1.6 Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.
Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records".

Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

    1054112
    1154112
    1254112
    1354112

Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the average trip duration in January?

    15.16
    19.16
    24.16
    29.16

Data preparation

Check the distribution of the duration variable. There are some outliers.

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop?
Q3. Missing values

The features we'll use for our model are the pickup and dropoff location IDs.

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

    53%
    63%
    73%
    83%

Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    Turn the dataframe into a list of dictionaries
    Fit a dictionary vectorizer
    Get a feature matrix from it

What's the dimensionality of this matrix? (The number of columns).

    2
    152
    352
    525
    725

Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

    Train a plain linear regression model with default parameters
    Calculate the RMSE of the model on the training data

What's the RMSE on train?

    5.52
    10.52
    15.52
    20.52

Q6. Evaluating the model

Now let's apply this model to the validation dataset (Feb 2021).

What's the RMSE on validation?

    6.01
    11.01
    16.01
    21.01


In [1]:
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')

In [3]:
old_len = len(df)

In [4]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

In [5]:
df.duration.mean()

19.1672240937939

In [6]:
df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

In [7]:
categorical = ['PUlocationID', 'DOlocationID']

df[categorical] = df[categorical].fillna(-1).astype('int')

In [8]:
df[categorical] = df[categorical].astype('str')

In [9]:
train_dicts = df[categorical].to_dict(orient='records')

In [10]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [11]:
X_train.shape

(1109826, 525)

In [12]:
y_train = df.duration.values

In [13]:
len(dv.feature_names_)

525

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [15]:
y_pred = lr.predict(X_train)

In [16]:
mean_squared_error(y_train, y_pred, squared=False)

10.528519107205765

In [17]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [18]:
df_val = read_data('./data/fhv_tripdata_2021-02.parquet')

In [19]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [20]:
X_val = dv.transform(val_dicts)

In [21]:
y_pred = lr.predict(X_val)

In [22]:
y_val = df_val.duration.values

In [23]:
mean_squared_error(y_val, y_pred, squared=False)

11.014283147378706